In [8]:
import requests
import os
import numpy as np
base_url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
categories = ['circle', 'square', 'triangle', 'star', 'hexagon']
if not os.path.exists('quickdraw_data'):
    os.makedirs('quickdraw_data')
def download_and_save(category):
    url = base_url + f'{category}.npy'
    filename = os.path.join('quickdraw_data', f'{category}.npy')
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f'Downloaded {category}')
for category in categories:
    download_and_save(category)

print("Selected categories have been downloaded.")


Downloaded circle
Downloaded square
Downloaded triangle
Downloaded star
Downloaded hexagon
Selected categories have been downloaded.


In [9]:
import requests
import os
base_url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
categories = ['circle', 'square', 'triangle', 'star', 'hexagon']
if not os.path.exists('quickdraw_data'):
    os.makedirs('quickdraw_data')
def download_and_save(category):
    url = base_url + f'{category}.npy'
    filename = os.path.join('quickdraw_data', f'{category}.npy')
    if os.path.exists(filename):
        os.remove(filename)
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f'Downloaded {category}')
for category in categories:
    download_and_save(category)

print("Selected categories have been redownloaded.")


Downloaded circle
Downloaded square
Downloaded triangle
Downloaded star
Downloaded hexagon
Selected categories have been redownloaded.


In [11]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
categories = ['circle', 'square', 'triangle', 'star', 'hexagon']
X = []
y = []
for idx, category in enumerate(categories):
    file_path = os.path.join('quickdraw_data', f'{category}.npy')
    if os.path.exists(file_path):
        data = np.load(file_path, allow_pickle=True)  
        X.extend(data)
        y.extend([idx] * len(data))
X = np.array(X)
y = np.array(y)
X = X / 255.0
X = np.expand_dims(X, axis=-1)
X_reshaped = X.reshape(-1, 28, 28, 1)  
X_resized = []
for img in X_reshaped: 
    img_tensor = tf.convert_to_tensor(img)
    img_resized = tf.image.resize(img_tensor, (32, 32)).numpy()
    X_resized.append(img_resized)
X = np.array(X_resized)
y = to_categorical(y, num_classes=len(categories))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.repeat(X_train, 3, axis=-1)
X_val = np.repeat(X_val, 3, axis=-1)
print("Preprocessing complete.")
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

Preprocessing complete.
Training data shape: (520996, 32, 32, 3)
Validation data shape: (130249, 32, 32, 3)


In [12]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



In [13]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)  
predictions = Dense(len(categories), activation='softmax')(x)  
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


94765736/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                         

In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]
)




Epoch 1/10
16282/16282 [==============================] - 1003s 62ms/step - loss: 0.2713 - accuracy: 0.9054 - val_loss: 0.2452 - val_accuracy: 0.9177
Epoch 2/10
16282/16282 [==============================] - 999s 61ms/step - loss: 0.2238 - accuracy: 0.9219 - val_loss: 0.2043 - val_accuracy: 0.9296
Epoch 3/10
16282/16282 [==============================] - 1008s 62ms/step - loss: 0.2110 - accuracy: 0.9262 - val_loss: 0.2109 - val_accuracy: 0.9251
Epoch 4/10
16282/16282 [==============================] - 1001s 61ms/step - loss: 0.2030 - accuracy: 0.9292 - val_loss: 0.2015 - val_accuracy: 0.9301
Epoch 5/10
16282/16282 [==============================] - 1007s 62ms/step - loss: 0.1979 - accuracy: 0.9307 - val_loss: 0.1889 - val_accuracy: 0.9349
Epoch 6/10
16282/16282 [==============================] - 998s 61ms/step - loss: 0.1938 - accuracy: 0.9323 - val_loss: 0.2111 - val_accuracy: 0.9251
Epoch 7/10
16282/16282 [==============================] - 995s 61ms/step - loss: 0.1912 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
import os
model_save_path = '/content/drive/My Drive/resnet50_quickdraw_model.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to /content/drive/My Drive/resnet50_quickdraw_model.h5
